In [1]:
import torch
from torch import nn

In [2]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(2, 4)
net(X)

tensor([[0.2884],
        [0.2127]], grad_fn=<AddmmBackward0>)

In [3]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.2339, -0.2849,  0.0207, -0.1780, -0.2391,  0.1191,  0.1018,  0.3012]])), ('bias', tensor([0.1485]))])


In [4]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.1485], requires_grad=True)
tensor([0.1485])


In [6]:
net[2].weight.grad == None

True

访问所有参数

In [11]:
# param：展示 Parameter containing
# name：参数名称
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [21]:
# 访问网络参数方式
# 键：每层的参数名
# 值：参数值
net.state_dict()['2.bias'].data

tensor([0.1485])

嵌套块参数

In [22]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block{i}', block1())
    return net

In [23]:
rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.3760],
        [-0.3760]], grad_fn=<AddmmBackward0>)

In [25]:
# 打印模型结构（简单模型）
print(rgnet)

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [26]:
# 通过索引访问
rgnet[0][1][0].bias.data

tensor([ 0.3325, -0.3274,  0.4777,  0.3070, -0.1311, -0.4716,  0.2809,  0.3000])

参数初始化

In [27]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

In [28]:
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0075,  0.0012, -0.0044,  0.0107]), tensor(0.))

In [29]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

In [30]:
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [31]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

In [32]:
# 42 宇宙的答案
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

In [34]:
net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([[-0.1785, -0.1375, -0.4716,  0.2510],
        [ 0.2351,  0.5025, -0.2328, -0.2757],
        [ 0.3915,  0.4771,  0.5052,  0.1471],
        [-0.3869, -0.5484, -0.2778, -0.6292],
        [-0.1304, -0.3235,  0.4672, -0.2836],
        [-0.1586,  0.2096, -0.2315, -0.5407],
        [-0.5416, -0.1582, -0.5251,  0.0906],
        [ 0.0662,  0.1616, -0.0617,  0.3769]])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


自定义初始化

In [41]:
def my_init(m):
    if type(m) == nn.Linear:
        # [0]：只打印 weight 信息
        print('Init', *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

In [42]:
net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 8.2833, -6.2322, -0.0000,  5.7258],
        [ 0.0000,  7.3483, -0.0000,  0.0000]], grad_fn=<SliceBackward0>)

In [43]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, -5.2322,  1.0000,  6.7258])

两个层的参数绑定，参数共享

In [44]:
shared = nn.Linear(8, 8)
net = nn.Sequential(
    nn.Linear(4, 8), nn.ReLU(),
    shared, nn.ReLU(),
    shared, nn.ReLU(),
    nn.Linear(8, 1)
                    )

In [46]:
net(X)
# 他们是同一个对象
# 反向传播时候，他们的梯度会加在一起，pytorch反向传播时候梯度会累加，只有每一次所有反向传播完毕后，梯度才清0
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
